### Maize Leaf Disease Classification

In [1]:
import PIL
import numpy as np
import os
import tensorflow as tf
import cv2
import keras
import pathlib
import glob
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import numpy

### Loading the Images

In [2]:
data_dir=pathlib.Path(os.getcwd()+"/data/")
maize_leaf_disease_dict={
    "Blight":list(glob.glob(f"{data_dir}/Blight/*.jpg")),
    "Common_Rust":list(glob.glob(f"{data_dir}/Common_Rust/*.jpg")),
    "Gray_Leaf_Spot":list(glob.glob(f"{data_dir}/Gray_Leaf_Spot/*.jpg")),
    "Healthy":list(glob.glob(f"{data_dir}/Healthy/*.jpg"))
}
maize_leaf_disease_labels={
    "Blight":0,
    "Common_Rust":1,
    "Gray_Leaf_Spot":2,
    "Healthy":3,
}

In [3]:
print(len(maize_leaf_disease_dict["Blight"]))
print(len(maize_leaf_disease_dict["Common_Rust"]))
print(len(maize_leaf_disease_dict["Gray_Leaf_Spot"]))
print(len(maize_leaf_disease_dict["Healthy"]))

1145
1306
574
1162


### Turning images into pixels

In [4]:
x,y=[],[]
for augmented_original,images in maize_leaf_disease_dict.items():
    for image in images:
        img=cv2.imread(str(image))#Convert image to pixels
        resized_img=cv2.resize(img,(180,180))#Images to be of same size
        x.append(resized_img)#Pixels
        y.append(maize_leaf_disease_labels[augmented_original])#Label Random 0,1
print(x[0])
print(y[:5])
x=np.array(x)
y=np.array(y)

[[[ 29  86  77]
  [ 36  84  69]
  [ 28  84  60]
  ...
  [  1 110  84]
  [  1 103  75]
  [  2 109  86]]

 [[ 19  83  66]
  [ 24  79  58]
  [ 27  87  62]
  ...
  [  3 115  87]
  [  3 111  83]
  [  8 109  86]]

 [[ 25  85  71]
  [ 35  87  73]
  [ 30  85  69]
  ...
  [  5 120  95]
  [  4 118  96]
  [ 14 121 103]]

 ...

 [[ 26 135  96]
  [ 34 143 117]
  [ 20 136 105]
  ...
  [ 15 125  88]
  [ 16 124  85]
  [ 12 123  81]]

 [[ 22 131  96]
  [ 25 134 109]
  [ 17 133 106]
  ...
  [ 10 122  83]
  [ 11 116  79]
  [ 11 118  80]]

 [[ 20 131  97]
  [ 30 139 114]
  [ 26 140 119]
  ...
  [ 15 124  84]
  [ 14 117  81]
  [ 13 113  75]]]
[0, 0, 0, 0, 0]


### Test, Train Split

In [5]:
x_train,x_test,y_train, y_test=train_test_split(x,y,test_size=0.3,random_state=0) #Splitting
print(len(x_train))
print(len(x_test))

2930
1257


### Scaling

In [6]:
x_train_scaled=x_train/255
x_test_scaled=x_test/255
print(x_train_scaled[0])

[[[0.3372549  0.65882353 0.49019608]
  [0.34901961 0.6627451  0.49411765]
  [0.37254902 0.67058824 0.50588235]
  ...
  [0.43529412 0.70588235 0.57647059]
  [0.4745098  0.74509804 0.61568627]
  [0.4745098  0.74509804 0.61568627]]

 [[0.3254902  0.64705882 0.47843137]
  [0.34509804 0.65882353 0.49019608]
  [0.35686275 0.65490196 0.49019608]
  ...
  [0.44313725 0.70980392 0.58431373]
  [0.47058824 0.7372549  0.61176471]
  [0.47058824 0.73333333 0.60784314]]

 [[0.3372549  0.65098039 0.48235294]
  [0.35686275 0.66666667 0.49803922]
  [0.34901961 0.64313725 0.47843137]
  ...
  [0.47058824 0.73333333 0.60784314]
  [0.49411765 0.75686275 0.63137255]
  [0.49411765 0.75686275 0.63137255]]

 ...

 [[0.38431373 0.5254902  0.47843137]
  [0.42745098 0.56078431 0.51764706]
  [0.36862745 0.50588235 0.45882353]
  ...
  [0.54509804 0.74117647 0.70980392]
  [0.54901961 0.7372549  0.70588235]
  [0.56862745 0.75294118 0.71764706]]

 [[0.40392157 0.54509804 0.49803922]
  [0.31372549 0.45098039 0.40784314]


### Creating the CNN model

In [7]:
num_classes=4
model=Sequential([
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128,activation="relu"),
    layers.Dense(num_classes)
])

In [8]:
len(model.layers)

9

### Compiling the Model

In [9]:
model.compile(optimizer="adam",
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"]
             )

### Training the CNN Model

In [10]:
model.fit(x_train_scaled,y_train,epochs=30)

Epoch 1/30
92/92 [==============================] - 43s 453ms/step - loss: 0.5464 - accuracy: 0.7785
Epoch 2/30
92/92 [==============================] - 45s 493ms/step - loss: 0.3265 - accuracy: 0.8659
Epoch 3/30
92/92 [==============================] - 45s 489ms/step - loss: 0.2479 - accuracy: 0.8969
Epoch 4/30
92/92 [==============================] - 44s 483ms/step - loss: 0.2146 - accuracy: 0.9102
Epoch 5/30
92/92 [==============================] - 48s 518ms/step - loss: 0.1927 - accuracy: 0.9205
Epoch 6/30
92/92 [==============================] - 50s 549ms/step - loss: 0.1407 - accuracy: 0.9502
Epoch 7/30
92/92 [==============================] - 57s 617ms/step - loss: 0.1430 - accuracy: 0.9529
Epoch 8/30
92/92 [==============================] - 45s 486ms/step - loss: 0.1080 - accuracy: 0.9577
Epoch 9/30
92/92 [==============================] - 49s 530ms/step - loss: 0.0602 - accuracy: 0.9802
Epoch 10/30
92/92 [==============================] - 46s 496ms/step - loss: 0.0567 - accura

### Testing the CNN Model

In [11]:
model.evaluate(x_test_scaled,y_test)

40/40 [==============================] - 6s 137ms/step - loss: 0.7148 - accuracy: 0.8870


[0.7148211002349854, 0.8870326280593872]

### Loading the test images

In [12]:
test_images=list(glob.glob(f"{os.getcwd()}/test_images/*.jpg"))
print(len(test_images))
t=[]
for image in test_images:
    img=cv2.imread(str(image))
    resized_img=cv2.resize(img,(180,180))
    t.append(resized_img)
t=np.array(t)
t_scale=t/255
print(t_scale)

16
[[[[0.65882353 0.76470588 0.78039216]
   [0.68627451 0.76470588 0.76470588]
   [0.7254902  0.78039216 0.77254902]
   ...
   [0.40392157 0.50980392 0.66666667]
   [0.34901961 0.44313725 0.59215686]
   [0.19607843 0.29019608 0.43137255]]

  [[0.60392157 0.70196078 0.71764706]
   [0.59607843 0.66666667 0.66666667]
   [0.62352941 0.67843137 0.66666667]
   ...
   [0.40392157 0.5254902  0.6745098 ]
   [0.34509804 0.45098039 0.59607843]
   [0.2        0.30196078 0.44313725]]

  [[0.60784314 0.69803922 0.70588235]
   [0.57647059 0.64313725 0.63529412]
   [0.60392157 0.65490196 0.63921569]
   ...
   [0.38039216 0.5254902  0.66666667]
   [0.31764706 0.43921569 0.58039216]
   [0.2        0.30588235 0.44313725]]

  ...

  [[0.3254902  0.41176471 0.3372549 ]
   [0.39607843 0.47843137 0.39607843]
   [0.34509804 0.41960784 0.3372549 ]
   ...
   [0.30980392 0.45490196 0.60784314]
   [0.30588235 0.42745098 0.58039216]
   [0.38039216 0.4745098  0.63137255]]

  [[0.36078431 0.45098039 0.36470588]
   [

### Prediction

In [13]:
prediction=model.predict(t_scale)
score=tf.nn.softmax(prediction[7]) #Picks the set of values in the tensor and picks the biggest on
print(score)
print(np.argmax(score)) 

tf.Tensor([1.5839742e-32 1.0000000e+00 1.7314021e-29 0.0000000e+00], shape=(4,), dtype=float32)
1


### Saving the model

In [14]:
model.save("image_classification") 

INFO:tensorflow:Assets written to: image_classification\assets
